In [1]:
import pandas as pd
import numpy as np
import datetime

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select, CustomJS, TapTool
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.palettes import Blues4
from bokeh.plotting import figure, show,output_file,save
from bokeh.io import output_notebook
output_notebook()

import locale
locale.setlocale(locale.LC_TIME, "ru_RU")

Loading BokehJS ...

'ru_RU'

In [ ]:
df = pd.read_csv('data.csv')
df.set_index(pd.to_datetime(df['time']),inplace=True)
df.drop(['time'], axis=1, inplace=True)
df.index = df.index+ pd.DateOffset(hours=3)

In [3]:
def plot_hist(df_row):
    hist_data = df_row.to_frame()
    date = df_row.name.strftime('%d %B %H:%M')

    hist_data.columns=['votes']

    hist_data.sort_values(by='votes', ascending=False, inplace=True)
    hist_data['rank'] = hist_data.reset_index().index
    hist_data['rank'] = hist_data['rank']+1
    hist_data.sort_values(by='votes', ascending=True, inplace=True)

    source = ColumnDataSource(hist_data)
    names = hist_data.index.tolist()
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save,hover"

    TOOLTIPS = [
        ("МЕСТО", "@rank"),
        ("КАНДИДАТ","@index"),
        ("ГОЛОСОВ", "@votes"),
    ]
    p = figure(y_range=names,tools=TOOLS, tooltips=TOOLTIPS,plot_width=800, plot_height=1200,sizing_mode='scale_width' )
    # p.vbar(x='index', top='votes', width=0.9, source=source,alpha=0.7, hover_fill_alpha = 1.0)
    p.hbar(y='index', right='votes', height=0.9, source=source, alpha=0.7, hover_fill_alpha = 1.0, left=1)
    
    title = f'Данные по состоянию на {date}'
    p.title.text=title
    p.title.text_font_size = "25px"
    return p
df_row = df.loc['2020-04-25 18:05:29']
pp = plot_hist(df_row)
show(pp)

In [12]:
hist_data = df_row.to_frame()
date = df_row.name.strftime('%d %B %H:%M')

hist_data.columns=['votes']

hist_data.sort_values(by='votes', ascending=False, inplace=True)
hist_data['rank'] = hist_data.reset_index().index
hist_data['rank'] = hist_data['rank']+1
hist_data.sort_values(by='votes', ascending=True, inplace=True)

sourcebars = ColumnDataSource(hist_data)
names = hist_data.index.tolist()
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,hover,tap"

TOOLTIPS = [
    ("МЕСТО", "@rank"),
    ("КАНДИДАТ","@index"),
    ("ГОЛОСОВ", "@votes"),
]
p1 = figure(y_range=names,tools=TOOLS, tooltips=TOOLTIPS,plot_width=800, plot_height=1200,sizing_mode='scale_width' )
bars = p1.hbar(y='index', right='votes', height=0.9, source=sourcebars, alpha=0.7, hover_fill_alpha = 1.0, left=1)


# source_bars = ColumnDataSource({'x': [1, 2, 3], 'y': [2, 4, 1] , 'colnames': ['A', 'B', 'C']})
# lines_y = [np.random.random(5) for i in range(3)]

# plot1 = figure(tools = 'tap')
# bars = plot1.vbar(x = 'x', top = 'y', source = source_bars, bottom = 0, width = 0.5)

df['y']=0
sourcelines = ColumnDataSource(df)
# lines_y = sourcelines
p2 = figure()
lines = p2.line(x = 'time', y = 'y', source = sourcelines)
lines.visible = False

code = '''if (cb_data.source.selected.indices.length > 0){
            lines.visible = true;
            console.log('here');
            var selected_index = cb_data.source.selected;
            lines.data_source.data['y'] = lines_y[selected_index]
            lines.data_source.change.emit(); 
          }'''

plots = row(p1, p2)
p1.select(TapTool).callback = CustomJS(args = {'lines': lines, 'lines_y': sourcelines}, code = code)
show(plots)